<h2>Final Exam</h2>
You are a stock analyst working in the research department of a small<br>
hedge fund.  Your job is to analyze the returns of a pool of securities<br>
and back test the quants strategy of buys and sells.  You need<br>
to load the 4 securities and test their strategy.<br>
files:<br>
INTC - Intel Corporation<br>
AMD - Advanced Micro Devices<br>
AAPL - Apple<br>
FB - Meta Platforms, Inc.<br>
corporations = {'INTC': 'Intel Corporation',<br>
'AMD':'Advanced Micro Devices',<br>
'AAPL':'Apple',<br>
'FB':'Meta Platforms, Inc.'}<br>
<br>
NOTE: All trades use the closing price!!!!

<h2>Step 1</h2>
<h4>5 points</h4>
Create a directory and place all of the csv files into it<br>
Create the variable proj_dir = '  <your directory> '<br>
db_name = 'stock_analysis'

In [1]:
import sqlite3
from datetime import date
from random import randint
from pathlib import Path

# Directory containing CSV files
proj_dir = Path('securities')
# Name of database
db_name = 'stock_analysis'

<h2>Step 2</h2>
<h4>5 points</h4>
Create a database called 'Stock_Analysis'

In [2]:
# Create the database
connection = sqlite3.connect(f'{db_name}.db')
cursor = connection.cursor()

# Commit changes and close connection
connection.commit()
connection.close()

<h2>Step 3</h2>
<h4>15 points</h4>
Create the following tables:<br>
1 - security_master<br>
symbol<br>
trade_date<br>
open_price<br>
high_price<br>
low_price<br>
adj_close<br>
close_price<br>
volume<br>
<br>
2 - security_info<br>
symbol<br>
name  Max of 25 characters<br>
research_notes Max of 255 characters<br>
<br>
3 - security_log<br>
transaction_id  Character max 20 alpha numeric<br>
trade_date<br>
symbol<br>
trans_type   - (will contain only 'BUY' or 'SELL')<br>
shares   (Will only contain whole numbers)<br>
cost - Will hold dollar amounts<br>
balance - Will hold dollar amounts<br>

In [3]:
# Connect to the database
connection = sqlite3.connect(f'{db_name}.db')
cursor = connection.cursor()

# Create the tables
cursor.executescript(
    '''
    create table if not exists security_master (
        symbol text,
        trade_date text,
        open_price real,
        high_price real,
        low_price real,
        adj_close real,
        close_price real,
        volume integer
    );
    create table if not exists security_info (
        symbol text,
        name text,
        research_notes text
    );
    create table if not exists security_log (
        transaction_id text,
        trade_date text,
        symbol text,
        trans_type text,
        shares integer,
        cost real,
        balance real
    );
    '''
)

# Commit changes and close connection
connection.commit()
connection.close()

<h2>Step 4</h2>
<h4>15 points</h4>
Load the security master table<br>
Your code should read whatever files that are csv files in the directory<br>
Use the dictionary below to populate the security_info table:<br>
corporations = {'INTC': 'Intel Corporation',
'AMD':'Advanced Micro Devices',
'AAPL':'Apple',
'FB':'Meta Platforms, Inc.'}

In [4]:
# Connect to the database
connection = sqlite3.connect(f'{db_name}.db')
cursor = connection.cursor()

# Corporation codes and names
corporations = {
    'INTC': 'Intel Corporation',
    'AMD':'Advanced Micro Devices',
    'AAPL':'Apple',
    'FB':'Meta Platforms, Inc.'
}

# Truncate the tables for fresh start on each run
cursor.executescript(
    '''
    delete from security_master;
    delete from security_info;
    '''    
)

# Open files, extract data, and populate security_master table
for code in corporations:
    with open(f'{proj_dir}/{code}.csv', 'r') as data_file:
        # List of lines without newline character
        lines = [line[:-1] for line in data_file.readlines()[1:]]
    # Pull data from the lines
    for line in lines:
        # Add the corp_code to the data frame
        values = [code] + line.split(',')
        # Correcting data types of the values
        data_types = [str, str, float, float, float, float, float, int]
        frame = [dt(v) for dt,v in zip(data_types, values)]
        # Insert the data frame into the table
        cursor.execute(
            '''
            insert into security_master values
            (?,?,?,?,?,?,?,?)
            ''',
            tuple(frame)
        )

# Populate the security_info table using the corporations dictionary
for code,name in corporations.items():
    cursor.execute(
        '''
        insert into security_info values
        (?,?,?)
        ''',
        (code,name,'')
    )

# Commit changes and close connection
connection.commit()
connection.close()

<h2>Step 5</h2>
<h4>10 points</h4>
Print the symbol,name, and number of records of every security using 1 SQL statement<br>

In [5]:
# Connect to the database
connection = sqlite3.connect(f'{db_name}.db')
cursor = connection.cursor()

# SQL query for each corporation
for corp in corporations:
    cursor.execute(
        f'''
        select security_info.symbol,security_info.name,count(security_master.symbol)
        from security_master
        join security_info on security_info.symbol=security_master.symbol
        where security_master.symbol=:code
        ''',
        {'code': corp}
    )
    print(*cursor.fetchall())

# Commit changes and close connection
connection.commit()
connection.close()

('INTC', 'Intel Corporation', 754)
('AMD', 'Advanced Micro Devices', 754)
('AAPL', 'Apple', 754)
('FB', 'Meta Platforms, Inc.', 754)


<h2>Step 6</h2>
<h4>10 points</h4>
Print the average closing share price for each security in the year 2019<br>

In [6]:
# Connect to the database
connection = sqlite3.connect(f'{db_name}.db')
cursor = connection.cursor()

# SQL query for each corporation
for corp in corporations:
    cursor.execute(
        f'''
        select symbol,avg(close_price)
        from security_master
        where symbol=:code and strftime('%Y', date(trade_date))='2019'
        ''',
        {'code': corp}
    )
    print(*cursor.fetchall())

# Commit changes and close connection
connection.commit()
connection.close()

('INTC', 48.324832607142824)
('AMD', 29.94107146031748)
('AAPL', 50.99662557936508)
('FB', 181.63749959523804)


<h2>Step 7</h2>
<h4>5 points</h4>
Print the max and min closing share price for each security in the year 2020<br>

In [7]:
# Connect to the database
connection = sqlite3.connect(f'{db_name}.db')
cursor = connection.cursor()

# SQL query for each corporation
for corp in corporations:
    cursor.execute(
        f'''
        select symbol,min(close_price),max(close_price)
        from security_master
        where symbol=:code and strftime('%Y', date(trade_date))='2020'
        ''',
        {'code': corp}
    )
    print(*cursor.fetchall())

# Commit changes and close connection
connection.commit()
connection.close()

('INTC', 42.636486, 65.120239)
('AMD', 38.709999, 97.120003)
('AAPL', 55.402668, 135.852524)
('FB', 146.009995, 303.910004)


<h2>Step 8</h2>
<h4>15 points</h4>
Alter the trading strategy below to update the security_log table<br>
transaction_id = YYYYMMDD_symbol_random number, 100 up to 999<br>
trade_date <br>
symbol <br>
trans_type BUY OR SELL<br>
shares number of shares<br>
cost =  -1 * close_price * shares if a BUY, close_price * shares if a SELL<br>
<br>
**(Code Redacted)**

In [8]:
# Connect to the database
connection = sqlite3.connect(f'{db_name}.db')
cursor = connection.cursor()

# Truncate the tables for fresh start on each run
cursor.execute('delete from security_log')

# Transaction data for AMD
cursor.execute(
    f'''
    select symbol,trade_date,close_price-open_price as diff,close_price
    from security_master
    where symbol='AMD'
    '''
)

# Counts for buying and selling
up_days = 0
down_days = 0
total_value = 0
total_shares = 0
share_size = 10000

for row in cursor.fetchall():
    symbol = row[0]
    trade_date = row[1]
    # Trade date object for the transaction_id
    d_obj = date(*[int(i) for i in trade_date.split('-')])
    y,m,d = str(d_obj.year),str(d_obj.month),str(d_obj.day)
    transaction_id =f'{y}_00{m}_00{d}_{row[0]}_{randint(100, 1000)}'
    # Track up and down days
    if row[2] > 0:
        up_days += 1
        down_days = 0
    else:
        up_days = 0
        down_days += 1
    # Sell when stock up for 5 days
    if up_days >= 5:
        down_days = 0
        if total_shares > 0:
            factor = 2 if total_shares - (share_size * 2) > 0 else 1
            shares_to_sell = share_size * factor
            profit =  shares_to_sell * row[3]
            total_shares -= shares_to_sell
            total_value += profit
            # Insert sells to security_log
            cursor.execute(
                '''
                insert into security_log values
                (?,?,?,?,?,?,?)
                ''',
                (transaction_id, trade_date, symbol, 'SELL', shares_to_sell, round(profit, 2), total_shares)
            )
    # Buy when stock is down for 5 days
    if down_days >= 5:
        up_days = 0
        down_days = 0
        cost = -1 * share_size * row[3]
        total_value += cost
        total_shares += share_size
        # Insert buys to security_log
        cursor.execute(
            '''
            insert into security_log values
            (?,?,?,?,?,?,?)
            ''',
            (transaction_id, trade_date, symbol, 'BUY', share_size, round(cost, 2), total_shares)
        )

# Commit changes and close connection
connection.commit()
connection.close()

<h2>Step 9</h2>
<h4>10 points</h4>
If you missed the update on step 8 above manual run the following inserts<br>
Then return the symbol,stock price,trans_type For all 2020 trades<br>
**(inserts not needed)**

In [9]:
# Connect to the database
connection = sqlite3.connect(f'{db_name}.db')
cursor = connection.cursor()

# Querying 2020 trade log
cursor.execute(
    f'''
    select symbol,abs(cost/shares) as stock_price,trans_type
    from security_log
    where strftime('%Y', date(trade_date))='2020'
    '''
)
for row in cursor.fetchall():
    print(row)

# Commit changes and close connection
connection.commit()
connection.close()

('AMD', 49.880001, 'BUY')
('AMD', 81.989998, 'SELL')
('AMD', 81.559998, 'BUY')
('AMD', 93.160004, 'BUY')


<h2>Step 10</h2>
<h4>10 points</h4>
If you bought 100,000 shares of FB the first trading day of 2020 and sold<br>
100,000 shares on the last day of 2020. How much would you have gained or lost?

In [10]:
# Connect to the database
connection = sqlite3.connect(f'{db_name}.db')
cursor = connection.cursor()

# Sub-query to get first and last trade days of 2020 with stock prices for FB
cursor.execute(
    '''
    select * from (
        select close_price
        from security_master
        where strftime('%Y', date(trade_date))='2020'
        and symbol='FB'
        limit 1
    ),(
        select close_price
        from security_master
        where strftime('%Y', date(trade_date))='2020'
        and symbol='FB'
        order by trade_date desc
        limit 1
    )
    '''
)
for row in cursor.fetchall():
    earnings = row[1] - row[0]
print(f'Earnings for 2020: ${round(earnings, 2)}')

# Commit changes and close connection
connection.commit()
connection.close()

Earnings for 2020: $63.38


<h2>Bonus Question</h2>
<h4>25 points</h4>
Write a program that buys 10000 shares of INTC every time the volume<br>
is greater than 10% for the average daily volume for 10 days and sell 10000 shares <br>
when the average volume for the past 5 days.  The initial buy should be when the shares <br>
have had 5 consecutive positive trading days